In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen = True)
class DataingestionConfig:
    root_dir:Path
    source_url: str
    local_data_file : Path
    unzip_dir : Path
    


In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


[2025-11-13 12:57:29,718: INFO: __init__: Logging system is working correctly!]


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])




    def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataingestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,   # ✅ fixed lowercase key
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataingestionConfig):
        self.config = config

    def download_file(self): 
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded successfully!")
            logger.info(f"Download info: \n{headers}")
        else:
            file_size = get_size(Path(self.config.local_data_file))
            logger.info(f"File already exists. Size: {file_size}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the specified directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")


In [10]:
from TextSummarizer.config.configuration import ConfigurationManager

config = ConfigurationManager()


[2025-11-13 12:57:29,914: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2025-11-13 12:57:29,918: INFO: common: YAML file 'config\params.yaml' loaded successfully]
[2025-11-13 12:57:29,921: INFO: common: Created directory at: artifacts]


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)



[2025-11-13 12:57:29,942: INFO: common: YAML file 'config\config.yaml' loaded successfully]
[2025-11-13 12:57:29,946: INFO: common: YAML file 'config\params.yaml' loaded successfully]
[2025-11-13 12:57:29,949: INFO: common: Created directory at: artifacts]
[2025-11-13 12:57:29,951: INFO: common: Created directory at: artifacts\data_ingestion]
[2025-11-13 12:57:34,432: INFO: 1755116386: artifacts/data_ingestion/data.zip downloaded successfully!]
[2025-11-13 12:57:34,442: INFO: 1755116386: Download info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C316:C09D4:1F19B:3CE16:69158864
Accept-Ranges: bytes
Date: Thu, 13 Nov 2025 07:27:33 GMT